In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb  9 02:04:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
!ls /content/drive/MyDrive/archive

 best_model_1028.52485385624.pth        best_model_2d_test.pth
 best_model_1036.3455625417614.pth      best_model_429.92960906408854.pth
 best_model_1045.4423704824671.pth      best_model.pth
 best_model_1046.6953950409832.pth      cubes444_Santa_states_and_n_best_moves.csv
 best_model_1048.3806560596204.pth      empty.txt
 best_model_1049.3992924525917.pth     'Heis 3 gens data'
 best_model_1049.4389572528742.pth      HeisenbergComputations.sage
 best_model_1056.1936967242325.pth      puzzle_info.csv
 best_model_1066.2490914415823.pth      puzzles_best_scores_numeric_states_etc.csv
 best_model_1069.0997380974331.pth     'Rationality of growths of groups Ho.pdf'
 best_model_1081.9982093330839.pth      UT4_growth
 best_model_1082.7521646249431.pth      UT4_states_and_distances
 best_model_1084.7952571798976.pth      UT5_growth
 best_model_1086.6350008167417.pth      UT5_states_and_distances
 best_model_1088.3257784321752.pth      UT6_growth
 best_model_1088.901281108623.pth       UT6_st

In [ ]:
from tqdm.notebook import tqdm
import time

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import torch
import math

if torch.cuda.is_available():
    device = torch.device("cuda")
    dtype = torch.float
else:
    device = torch.device("cpu")
    dtype = torch.float#int16
    dtype = torch.int16

print(dtype)
print(device)


torch.float32
cuda


In [ ]:
# dimension = 10
# dic={5: 20,
#      10: 140,
#     20:	250,
# 21:	273,
# 22:	297,
# 23:	322,
# 24:	348,
# 25:	375,
# 26:	403,
# 27:	432,
# 28:	462,
# 29:	493,
# 30:	525,
# 31:	558,
# 32:	592,
# 35:	700,
# 37:	777,
# 40:	900,
# 45:	1125,
# 50:	1375,
# 60:	1950,
# 70:	2625,
# 80:	3400,
# 90:	4275,
# 100: 5250,
# 150:	11625,
# 200:	20500,
# 300:	45750}
# n_random_walk_length = dic[dimension]
# rate_of_diffusion_distance =1
# n_random_walk_length = n_random_walk_length*rate_of_diffusion_distance
# n_random_walks_to_generate_validation_fixed = 100
# n_random_walk_length = 20
# n_random_walks_to_generate = 1000

# def make_S_N(N):
#     return [
#         [1,0,]+[q+2 for q in range(N-2)]       ,
#                [q+1 for q in range(N-1)] + [0,],
#         [N-1,]+[q+0 for q in range(N-1)]       ,
#     ]
# list_generators = make_S_N(dimension)


In [ ]:
def random_walks(n_random_walk_length,  n_random_walks_to_generate):
      '''
      Output:
      returns X,y: X - array of states, y - number of steps rw achieves it

      Input:
      generators - generators (moves) to make random walks  (permutations),
          can be list of vectors or array with vstacked vectors
      n_random_walk_length - number of visited nodes, i.e. number of steps + 1
      n_random_walks_to_generate - how many random walks will run in parrallel
      rw_start - initial states for random walks - by default we will use 0,1,2,3 ...
          Can be vector or array
          If it is vector it will be broadcasted n_random_walks_to_generate times,
          If it is array n_random_walks_to_generate - input n_random_walks_to_generate will be ignored
              and will be assigned: n_random_walks_to_generate = rw_start.shape[0]

      '''
      state_size        = len(list_generators[0])
      n_generators      = len(list_generators  )
      dtype_generators = torch.int64
      tensor_generators = torch.tensor(   list_generators       , device = device, dtype = dtype_generators, )
      state_destination = torch.arange( state_size, device=device, dtype = dtype).reshape(-1,state_size)
      array_of_states = state_destination.to(device).view( 1, state_size  ).expand( n_random_walks_to_generate, state_size )

      # Output: X,y - states, y - how many steps we achieve them
      # Allocate memory:
      X = torch.zeros( n_random_walks_to_generate*n_random_walk_length , state_size, device=device, dtype = dtype )

      # First portion of data  - just our state_rw_start state  multiplexed many times
      X[:n_random_walks_to_generate,:] = array_of_states
      y        = torch.tensor(range(n_random_walks_to_generate*n_random_walk_length), device=device)//n_random_walks_to_generate
      IX_moves = torch.randint(0, n_generators, (n_random_walks_to_generate*n_random_walk_length-n_random_walks_to_generate,), dtype = torch.int32, device=device)

      # Technical to make array[ IX_array] we need  actually to write array[ range(N), IX_array  ]

      # Main loop

      argnr = (torch.arange(n_random_walks_to_generate, dtype=torch.long, device=device)*torch.ones((n_random_walks_to_generate,tensor_generators.shape[1]), dtype=torch.long, device=device).T).T

      for i_step in range(1,n_random_walk_length):
          a,b,c = (i_step-1)*n_random_walks_to_generate, (i_step-0)*n_random_walks_to_generate, (i_step+1)*n_random_walks_to_generate
          #X[ b:c, : ] = X[ a:b, : ][argnr, self.tensor_generators[IX_moves[ a:b ],:]]
          X[ b:c, : ] = torch.gather( X[ a:b, : ], 1, tensor_generators[IX_moves[ a:b ],:] )

      return X,y

In [ ]:
# X_test, y_test = random_walks(n_random_walk_length       = n_random_walk_length,
#                                                      n_random_walks_to_generate = n_random_walks_to_generate)
# test_dataset = MyIntSequenceDataset(X_test, y_test)
# # train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# i = 1
# for j in range(4):
#   print(j, X_test[i + j * 500])

0 tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], device='cuda:0')
1 tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], device='cuda:0')
2 tensor([1., 2., 3., 4., 5., 6., 7., 8., 9., 0.], device='cuda:0')
3 tensor([9., 0., 1., 2., 3., 4., 5., 6., 7., 8.], device='cuda:0')


In [ ]:
state_size = len(list_generators[0])
hash_vec = torch.randint(-(2**60), 2**60, (state_size,), dtype = torch.int64, device = device )
# hash_vec = torch.randint(0, 10**9, (state_size,), dtype = torch.int64, device = device )

def get_unique_states(states: torch.Tensor) -> torch.Tensor:
    '''
    Return matrix with unique rows for input matrix "states"
    I.e. duplicate rows are dropped.
    For fast implementation: we use hashing via scalar/dot product.
    Note: output order of rows is different from the original.
    '''
    # Note: that implementation is 30 times faster than torch.unique(states, dim = 0) - because we use hashes  (see K.Khoruzhii: https://t.me/sberlogasci/10989/15920)
    # Note: torch.unique does not support returning of indices of unique element so we cannot use it
    # That is in contrast to numpy.unique which supports - set: return_index = True


    # Hashing rows of states matrix:
    hashed = torch.sum(hash_vec * states, dim=1) # Compute hashes.
        # It is same as matrix product torch.matmul(hash_vec , states )
        # but pay attention: such code work with GPU for integers
        # While torch.matmul - does not work for GPU for integer data types,
        # since old GPU hardware (before 2020: P100, T4) does not support integer matrix multiplication

    # Sort
    hashed_sorted, idx = torch.sort(hashed)
    # Mask selects elements which are different from the consequite - that is unique elements (since vector is sorted on the previous step)
    mask = torch.concat((torch.tensor([True], device = device), hashed_sorted[1:] - hashed_sorted[:-1] > 0))
    return states[idx][mask]


class MatrixDatasetRegrSmall(Dataset):
    def __init__(self, X):
        self.X = torch.tensor(X, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = F.one_hot(self.X[idx], num_classes=self.X.shape[1]).float()
        return x.unsqueeze(0)

class MyIntSequenceDatasetSmall(Dataset):
    def __init__(self, X, is_regression=True):
        # Store integer inputs and targets
        dtype_y = torch.float32 if is_regression else torch.long
        self.X = torch.tensor(X, dtype=torch.long)  # shape (N, 96)

        self.is_regression = is_regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (input_sequence, target)
        return self.X[idx]

def make_predictions_on_array(model, array):
    model.eval()
    all_preds = []
    dataset = MyIntSequenceDatasetSmall(array)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    with torch.no_grad():
        for batch_X in loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())

    all_preds = np.array(all_preds)

    return all_preds



In [ ]:
# def beam_search_func():

#   n_scrambles_starting_state = 1000
#   n_gens = len(list_generators)
#   print('n_scrambles_starting_state:',n_scrambles_starting_state)

#   state_size = len(list_generators[0])
#   # destination state - "solved puzzle state"
#   state_destination = torch.arange( state_size, device=device, dtype = dtype)


#   # Scramble - generate state which will be start of beam search
#   state_start = state_destination
#   for k in range(n_scrambles_starting_state):
#       IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
#       state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ]


#   beam_width = 10_000

#   # Iabsolutenitialize array of states
#   array_of_states = state_start.view(1, state_size  ).clone().to(dtype).to(device)

#   i_step_max = 400
#   for i_step in range(1,i_step_max+1):

#       # Technical preparation: to apply permutations to array we need the trick:
#       # naively it is: array[ :, IX_array], but actually we need  to write array[ range(N)[:, np.newaxis], IX_array  ]
#       row_indices = np.arange( array_of_states.shape[0] )[:, np.newaxis]

#       # Apply all moves to all current states, all new states saved in array_of_states_new
#       array_of_states_new = torch.empty( (0,array_of_states.shape[1]) , device=device, dtype = dtype)
#       for move in list_generators:
#           array_of_states_tmp = array_of_states[row_indices,move]
#           array_of_states_new = torch.concatenate([array_of_states_new, array_of_states_tmp],axis = 0)

#       # Take only unique states
#       # surprise: THAT IS CRITICAL for beam search performance !!!!
#       # if that is not done - beam search  will not find the desired state - quite often
#       # The reason - essentianlly beam can degrade, i.e. can be populated by copy of only one state
#       # It is surprising that such degradation  happens quite often even for beam_width = 10_000 - but it is indeed so
#       array_of_states_new = get_unique_states(array_of_states_new)

#       # Check destination state found
#       vec_tmp = torch.all(array_of_states_new == state_destination, axis =1) # Compare state_destination and each row array_of_states
#       flag_found_destination = torch.any(vec_tmp) # Check for coincidence
#       if flag_found_destination:
#           print('Found destination state. ', 'i_step:', i_step, ' n_ways:', (vec_tmp).sum())
#           break

#       # ML-model inference - estimate distance of new states to the destination state
#       t0 = time.time()
#       if array_of_states_new.shape[0] > beam_width: # If we have not so many states - we take them all - no need for ML-model

#           # ML-model inference - estimate distance of new states to the destination state
#   #         y_pred = model.predict(array_of_states_new.cpu().numpy() )
#           y_pred = make_predictions_on_array(model, array_of_states_new.cpu().numpy() )

#           # Take only "beam_width" of the best states (i.e. most nearest to destination according to the model estimate)
#           idx = np.argsort(y_pred)[:beam_width]
#           array_of_states = array_of_states_new[idx,:]

#       else:
#           array_of_states = array_of_states_new
#       predict_time = time.time() - t0
#       print(i_step,'i_step', '%.3f'%predict_time, 'predict_time', array_of_states_new.shape, 'array_of_states_new.shape' )

#   print();
#   print('Search finished.', 'beam_width:', beam_width)
#   if flag_found_destination:
#       print(i_step, ' steps to destination state. Path found.')
#   else:
#       print('Path not found.')

In [ ]:

# def beam_search(state_start, model, beam_width =  10_000, i_step_max = 400, verbose = 0):
#     '''
#     Finds a path from the state_start to the destination state.

#     '''
#     # Initialize array of states
#     array_of_states = state_start.view(1, state_size  ).clone().to(dtype).to(device)
#     state_destination = torch.arange( state_size, device=device, dtype = dtype)
#     # i_step_max = 100
#     for i_step in range(1,i_step_max+1):

#         # Technical preparation: to apply permutations to array we need the trick:
#         # naively it is: array[ :, IX_array], but actually we need  to write array[ range(N)[:, np.newaxis], IX_array  ]
#         row_indices = np.arange( array_of_states.shape[0] )[:, np.newaxis]

#         # Apply all moves to all current states, all new states saved in array_of_states_new
#         array_of_states_new = torch.empty( (0,array_of_states.shape[1]) , device=device, dtype = dtype)
#         for move in list_generators:
#             array_of_states_tmp = array_of_states[row_indices,move]
#             array_of_states_new = torch.concatenate([array_of_states_new, array_of_states_tmp],axis = 0)

#         # Take only unique states
#         # surprise: THAT IS CRITICAL for beam search performance !!!!
#         # if that is not done - beam search  will not find the desired state - quite often
#         # The reason - essentianlly beam can degrade, i.e. can be populated by copy of only one state
#         # It is surprising that such degradation  happens quite often even for beam_width = 10_000 - but it is indeed so
#         array_of_states_new = get_unique_states(array_of_states_new)

#         # Check destination state found
#         vec_tmp = torch.all(array_of_states_new == state_destination, axis =1) # Compare state_destination and each row array_of_states
#         flag_found_destination = torch.any(vec_tmp).item() # Check for coincidence
#         if flag_found_destination:
#             if verbose >= 1:
#                 print('Found destination state. ', 'i_step:', i_step, ' n_ways:', (vec_tmp).sum())
#             break

#         # ML-model inference - estimate distance of new states to the destination state
#         t0 = time.time()
#         if array_of_states_new.shape[0] > beam_width: # If we have not so many states - we take them all - no need for ML-model

#             # ML-model inference - estimate distance of new states to the destination state
# #             y_pred = model.predict(array_of_states_new.cpu().numpy() )
#             y_pred = make_predictions_on_array(model, array_of_states_new.cpu().numpy() )
#             # Take only "beam_width" of the best states (i.e. most nearest to destination according to the model estimate)
#             idx = np.argsort(y_pred)[:beam_width]
#             array_of_states = array_of_states_new[idx,:]

#         else:
#             array_of_states = array_of_states_new
#         predict_time = time.time() - t0
#         if verbose >= 10:
#             print(i_step,'i_step', '%.3f'%predict_time, 'predict_time', array_of_states_new.shape, 'array_of_states_new.shape' )

#     if verbose >= 1:
#         print();
#         print('Search finished.', 'beam_width:', beam_width)
#         if flag_found_destination:
#             print(i_step, ' steps to destination state. Path found.')
#         else:
#             print('Path not found.')

#     return flag_found_destination, i_step

# def run_beam_search():
#     # %%time
#   n_scrambles_starting_state = 5
#   n_gens = len(list_generators)
#   state_destination = torch.arange( state_size, device=device, dtype = dtype)
#   # Scramble - generate state which will be start of beam search
#   state_start = state_destination
#   for k in range(n_scrambles_starting_state):
#       IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
#       state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ]


#   flag_found_destination, i_step = beam_search(state_start, model, beam_width =  10, i_step_max = 200,
#                                               verbose = 100)


In [ ]:
# def full_stats():
#   # n_scrambles_starting_state = 1000
#   # print('n_scrambles_starting_state:',n_scrambles_starting_state)

#   fig = plt.figure(figsize = (20,8))

#   for beam_width in [1, 10]:

#       str_inf = 'Catboost + beam_width ' + str(beam_width)

#       df_stat = pd.DataFrame()

#       for i_trial in range(10):
#           if i_trial < 25:
#               #print()
#               print('trial:', i_trial)
#           # destination state - "solved puzzle state"
#           state_destination = torch.arange( state_size, device=device, dtype = dtype)

#           #state_start

#           prm_name = 'n_scramble '
#           list_prm_value = list(range(1,21))
#           for prm_value in list_prm_value:#  5_000,10_000, 30_000]:#, 100_000]:
#               n_scrambles_starting_state = prm_value

#               # Scramble - generate state which will be start of beam search
#               state_start = state_destination
#               for k in range(n_scrambles_starting_state):
#                   IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
#                   state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ]

#               t0 = time.time()
#               flag_found_destination, i_step = beam_search(state_start, model, beam_width =  beam_width, i_step_max = 200,
#                                                           verbose = 0)
#               if i_trial < 1:
#                   print('Found:',flag_found_destination,'steps:', i_step,'beam_width:', beam_width,
#                         'time: %.1f secs'%(time.time()-t0))

#               df_stat.loc[i_trial,'Path length.  '+prm_name+str(prm_value)] =  i_step
#               df_stat.loc[i_trial,'Solution found. '+prm_name +str(prm_value)] =  int(flag_found_destination)
#               df_stat.loc[i_trial,'Time. '+prm_name+str(prm_value)] =  np.round(time.time()-t0,1)

#       df_stat.to_csv('stat_beam_search_'+prm_name+'_'+str_inf+'.csv')
#       display(df_stat)
#       display(df_stat.describe().round(3).T)
#       display(df_stat.describe().round(3).to_csv('aggregated_stat_beam_search_'+prm_name+'_'+str_inf+'.csv'))


#       df_loc = df_stat.describe()
#       dat_loc = []
#       col_key = 'Solution found'
#       for col in df_stat.columns:
#           if not( col_key in col) : continue
#           dat_loc.append(df_loc.loc['mean', col] )

#       plt.plot(list_prm_value, dat_loc, '*-',label = str_inf)
#       plt.title(col_key , fontsize = 20  )
#       plt.legend(fontsize = 20 )
#       plt.savefig(col_key.replace('.', ' ') + '.png')
#   plt.grid()
#   plt.show()

In [ ]:
class MatrixDatasetRegr(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.int64)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        x = F.one_hot(self.X[idx], num_classes=self.X.shape[1]).float()
        return x.unsqueeze(0), self.y[idx]

class MyIntSequenceDataset(Dataset):
    def __init__(self, X, y, is_regression=True):
        # Store integer inputs and targets
        dtype_y = torch.float32 if is_regression else torch.long
        self.X = torch.tensor(X, dtype=torch.long)  # shape (N, 96)
        self.y = torch.tensor(y, dtype=dtype_y)     # shape (N,)

        self.is_regression = is_regression

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Return (input_sequence, target)
        return self.X[idx], self.y[idx]

# train_dataset = MyIntSequenceDataset(X_train, y_train)
# test_dataset = MyIntSequenceDataset(X_test, y_test)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
# model = CNNRegr().to(device)
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

def calculate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for batch_X, batch_y in data_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    rmse = np.sqrt(mean_squared_error(all_targets, all_preds))
    r2 = r2_score(all_targets, all_preds)

    return rmse, r2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
class Embedded1DCNN_Wider_Seq10(nn.Module):
    def __init__(self,
                 vocab_size=96,  # number of possible integer tokens
                 embed_dim=32,   # embedding dimension
                 out_dim=60,     # output dimension (e.g. 60 classes, or 1 for regression)
                 dropout_p=0.3
                ):
        """
        A 'wider' 1D CNN architecture that expects an input sequence length = 10.

        Steps:
          - Embedding -> dropout
          - Conv1 (128 channels), BN, ReLU, dropout -> pool
          - Conv2 (256 channels), BN, ReLU, dropout -> pool
          - Flatten -> FC (512) -> dropout -> FC (out_dim)

        After 2 pools (each with kernel_size=2), the sequence length goes:
          10 -> 5 -> 2.

        The final feature map is (batch_size, 256, 2),
        so fc_input_dim = 256 * 2 = 512.
        """
        super().__init__()
        self.out_dim = out_dim

        # Embedding
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.dropout_embed = nn.Dropout(p=dropout_p)

        # Convolution layers (wider than usual)
        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(128)
        self.dropout_conv1 = nn.Dropout(p=dropout_p)

        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(256)
        self.dropout_conv2 = nn.Dropout(p=dropout_p)

        # Pooling
        self.pool = nn.MaxPool1d(kernel_size=2)

        # After 2 pools on length=10 => 10 -> 5 -> 2
        # Final shape after conv2+pool: (batch_size, 256, 2)
        self.fc_input_dim = 256 * 2

        # Fully connected
        self.fc1 = nn.Linear(self.fc_input_dim, 512)  # large width
        self.dropout_fc1 = nn.Dropout(p=dropout_p)
        self.fc2 = nn.Linear(512, out_dim)

    def forward(self, x):
        """
        x: (batch_size, seq_len=10) integer tokens in [0..vocab_size-1].
        """
        # Embedding => (batch_size, 10, embed_dim)
        x = self.embedding(x)
        x = self.dropout_embed(x)

        # Permute for Conv1d => (batch_size, embed_dim, 10)
        x = x.permute(0, 2, 1)

        # Conv block 1
        x = self.conv1(x)               # (batch_size, 128, 10)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout_conv1(x)
        x = self.pool(x)                # (batch_size, 128, 5)

        # Conv block 2
        x = self.conv2(x)               # (batch_size, 256, 5)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout_conv2(x)
        x = self.pool(x)                # (batch_size, 256, 2)

        # Flatten => (batch_size, 256*2)
        x = x.view(x.size(0), -1)       # => (batch_size, 512)

        # Fully connected
        x = self.fc1(x)                 # => (batch_size, 512)
        x = self.dropout_fc1(x)
        x = F.relu(x)

        # Final output
        logits = self.fc2(x)           # => (batch_size, out_dim)
        if self.out_dim == 1:
          return logits.squeeze(1)
        else:
          return logits


In [ ]:
class Embedded1DCNN_Flexible(nn.Module):
    def __init__(self,
                 vocab_size=96,   # Number of integer tokens possible
                 embed_dim=32,    # Embedding dimension
                 out_dim=60,      # Number of output classes (or 1 for regression)
                 dropout_p=0.3
                ):
        """
        A flexible 1D CNN architecture with:
          - Embedding layer
          - 2 convolutional blocks (Conv1d -> BN -> ReLU -> Dropout)
          - An adaptive average pool to reduce the sequence dimension to 1
          - Fully-connected layers for classification

        This approach can handle any input length >= 1 (e.g., 10, 20, 40),
        because the final 'nn.AdaptiveAvgPool1d(1)' automatically downscales
        to a length of 1 regardless of the initial sequence size.
        """
        super().__init__()
        self.out_dim = out_dim

        # Embedding for integer tokens
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim)
        self.dropout_embed = nn.Dropout(p=dropout_p)

        # ---- Convolutional Blocks ----
        # Block 1: in_channels=embed_dim -> out_channels=128
        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=128, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm1d(128)
        self.dropout_conv1 = nn.Dropout(p=dropout_p)

        # Block 2: in_channels=128 -> out_channels=256
        self.conv2 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm1d(256)
        self.dropout_conv2 = nn.Dropout(p=dropout_p)

        # ---- Adaptive Pooling ----
        # This layer will convert any (batch, 256, seq_len') to (batch, 256, 1).
        self.global_pool = nn.AdaptiveAvgPool1d(output_size=1)

        # ---- Fully-Connected Head ----
        # After pooling => shape (batch_size, 256, 1) => flatten => (batch_size, 256)
        self.fc1 = nn.Linear(256, 512)
        self.dropout_fc1 = nn.Dropout(p=dropout_p)

        # Final classification layer
        self.fc2 = nn.Linear(512, out_dim)

    def forward(self, x):
        """
        x shape: (batch_size, seq_len), each entry is an integer in [0..vocab_size-1].
        seq_len can be 10, 20, 40, etc.
        """
        # 1) Embedding => (batch_size, seq_len, embed_dim)
        x = self.embedding(x)
        x = self.dropout_embed(x)

        # Permute for Conv1d => (batch_size, embed_dim, seq_len)
        x = x.permute(0, 2, 1)

        # 2) Conv Block 1
        x = self.conv1(x)                # => (batch_size, 128, seq_len)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout_conv1(x)

        # 3) Conv Block 2
        x = self.conv2(x)                # => (batch_size, 256, seq_len)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout_conv2(x)

        # 4) Global Adaptive Pooling => (batch_size, 256, 1)
        x = self.global_pool(x)

        # Flatten => (batch_size, 256)
        x = x.view(x.size(0), -1)

        # 5) Fully-connected Layers
        x = self.fc1(x)                  # => (batch_size, 512)
        x = F.relu(x)
        x = self.dropout_fc1(x)

        # 6) Final classification => (batch_size, out_dim)
        logits = self.fc2(x)
        if self.out_dim == 1:
          return logits.squeeze(1)
        else:
          return logits

In [ ]:

# model = Embedded1DCNN_Flexible(out_dim=1).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.01, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.8, verbose=True)
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
# print(f'Total trainable parameters: {pytorch_total_params}')

Total trainable parameters: 0.246913


In [ ]:
def train(num_epochs=2):
  # Training loop
  train_rmse_list, train_r2_list = [], []
  test_rmse_list, test_r2_list = [], []
  loss_list = []


  best_test_rmse = float('inf')
  patience = 5
  no_improve = 0
  best_model_weights = copy.deepcopy(model.state_dict())

  X, _ = random_walks(n_random_walk_length       = n_random_walk_length,
                                                      n_random_walks_to_generate = n_random_walks_to_generate)
  print(f'{X.shape=}')
  del X, _

  for epoch in range(num_epochs):
      X_train, y_train = random_walks(n_random_walk_length       = n_random_walk_length,
                                                      n_random_walks_to_generate = n_random_walks_to_generate)

      train_dataset = MyIntSequenceDataset(X_train, y_train)
      train_loader = DataLoader(train_dataset, batch_size=256 * 2, shuffle=True)
      model.train()
      for batch_X, batch_y in train_loader:
          batch_X, batch_y = batch_X.to(device), batch_y.to(device)

          optimizer.zero_grad()
          outputs = model(batch_X)
          loss = criterion(outputs, batch_y)
          loss.backward()
          optimizer.step()

      # Calculate metrics for train and test data
      train_rmse, train_r2 = calculate_metrics(model, train_loader)
      test_rmse, test_r2 = calculate_metrics(model, test_loader)
      train_rmse_list.append(train_rmse)
      train_r2_list.append(train_r2)
      test_rmse_list.append(test_rmse)
      test_r2_list.append(test_r2)

      scheduler.step(test_rmse)

      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
      print(f'Train - RMSE: {train_rmse:.4f}, R²: {train_r2:.4f}')
      print(f'Test  - RMSE: {test_rmse:.4f}, R²: {test_r2:.4f}')
      # Early stopping
      if test_rmse < best_test_rmse:
          best_test_rmse = test_rmse
          no_improve = 0
          torch.save(model.state_dict(), f'/content/drive/MyDrive/archive/best_model_{best_test_rmse}.pth')
          best_model_weights = copy.deepcopy(model.state_dict())
      else:
          no_improve += 1
          if no_improve == patience:
              print("Early stopping triggered")
              break
  print('Finished Training')
  print(f'Best metrics: {min(test_rmse_list)=:.4f} {max(test_r2_list)=:.4f}')
  model.load_state_dict(best_model_weights)
  return model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list

In [ ]:
# dimension = 16
# dic={5: 20,
#      10: 140,
#      16: 180,
#     20:	250,
# 21:	273,
# 22:	297,
# 23:	322,
# 24:	348,
# 25:	375,
# 26:	403,
# 27:	432,
# 28:	462,
# 29:	493,
# 30:	525,
# 31:	558,
# 32:	592,
# 35:	700,
# 37:	777,
# 40:	900,
# 45:	1125,
# 50:	1375,
# 60:	1950,
# 70:	2625,
# 80:	3400,
# 90:	4275,
# 100: 5250,
# 150:	11625,
# 200:	20500,
# 300:	45750}
# n_random_walk_length = dic[dimension]
# rate_of_diffusion_distance =10
# n_random_walk_length = n_random_walk_length*rate_of_diffusion_distance
# n_random_walks_to_generate_validation_fixed = 100
# # n_random_walk_length = 40
# n_random_walks_to_generate = 1000

# def make_S_N(N):
#     return [
#         [1,0,]+[q+2 for q in range(N-2)]       ,
#                [q+1 for q in range(N-1)] + [0,],
#         [N-1,]+[q+0 for q in range(N-1)]       ,
#     ]
# list_generators = make_S_N(dimension)
# X_test, y_test = random_walks(n_random_walk_length       = n_random_walk_length,
#                                                      n_random_walks_to_generate = n_random_walks_to_generate)
# X_test.shape

torch.Size([1800000, 16])

In [ ]:
model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list = train(100)

X.shape=torch.Size([1800000, 16])
Epoch [1/100], Loss: 96527.3828
Train - RMSE: 332.1559, R²: 0.5914
Test  - RMSE: 288.1650, R²: -2496.4148
Epoch [2/100], Loss: 100970.5859
Train - RMSE: 341.1747, R²: 0.5689
Test  - RMSE: 349.6543, R²: -3675.9373
Epoch [3/100], Loss: 132497.2500
Train - RMSE: 331.2351, R²: 0.5936
Test  - RMSE: 331.7136, R²: -3308.2900
Epoch [4/100], Loss: 115068.4375
Train - RMSE: 333.8314, R²: 0.5872
Test  - RMSE: 350.4673, R²: -3693.0549
Epoch [5/100], Loss: 119225.0859
Train - RMSE: 327.6299, R²: 0.6024
Test  - RMSE: 326.8216, R²: -3211.4011
Epoch [6/100], Loss: 124021.6562
Train - RMSE: 333.2488, R²: 0.5887
Test  - RMSE: 307.0796, R²: -2835.0266
Epoch [7/100], Loss: 142338.9688
Train - RMSE: 334.8371, R²: 0.5848
Test  - RMSE: 272.6524, R²: -2234.7688
Epoch [8/100], Loss: 127898.1250
Train - RMSE: 344.3389, R²: 0.5609
Test  - RMSE: 276.8425, R²: -2304.0146
Epoch [9/100], Loss: 99430.2500
Train - RMSE: 344.9378, R²: 0.5593
Test  - RMSE: 298.8608, R²: -2685.2488
Epoch

In [ ]:
run_beam_search()

1 i_step 0.000 predict_time torch.Size([3, 40]) array_of_states_new.shape
2 i_step 0.000 predict_time torch.Size([7, 40]) array_of_states_new.shape
Found destination state.  i_step: 3  n_ways: tensor(1, device='cuda:0')

Search finished. beam_width: 10
3  steps to destination state. Path found.


In [ ]:
# 32  steps to destination state. Path found.
# 5  steps to destination state. Path found.

In [ ]:
def scramble_given_state_custom(list_generators, n_scrambles=1, state_to_scramble='01234...'):

    # Если начальное состояние задано по умолчанию, создаём массив из последовательных чисел
    if state_to_scramble == '01234...':
        state_size = len(list_generators[0])
        state_to_scramble = torch.arange(state_size)  # Используем тензор вместо массива

    # Приведение к тензору для упрощения операций
    if isinstance(state_to_scramble, (list, range)):
        state_to_scramble = torch.tensor(state_to_scramble)

    state_current = np.asarray(state_to_scramble.cpu())
    n = len(state_current)

    # Заранее выпишем саму перестановку p:
    # p[i] = индекс в state_current, из которого возьмётся элемент на позицию i после перестановки
    # Сначала p = [0, 1, 2, ..., n-1], а потом сделаем нужные перестановки:
    p = np.arange(n)
    # Обмен (1,2)
    p[0], p[1] = p[1], p[0]
    i = 2
    while i < n-i+1:
        # print(i, n-i+1)
        p[i], p[n-i+1] = p[n-i+1], p[i]
        i += 1

    # Теперь применим эту перестановку p n_scrambles раз
    for _ in range(n_scrambles):
        state_current = state_current[p]

    return torch.tensor(state_current, device=device)

In [ ]:
# state_size = len(list_generators[0])
# hash_vec = torch.randint(-(2**60), 2**60, (state_size,), dtype = torch.int64, device = device )
# # hash_vec = torch.randint(0, 10**9, (state_size,), dtype = torch.int64, device = device )

# def get_unique_states(states: torch.Tensor) -> torch.Tensor:
#     '''
#     Return matrix with unique rows for input matrix "states"
#     I.e. duplicate rows are dropped.
#     For fast implementation: we use hashing via scalar/dot product.
#     Note: output order of rows is different from the original.
#     '''
#     # Note: that implementation is 30 times faster than torch.unique(states, dim = 0) - because we use hashes  (see K.Khoruzhii: https://t.me/sberlogasci/10989/15920)
#     # Note: torch.unique does not support returning of indices of unique element so we cannot use it
#     # That is in contrast to numpy.unique which supports - set: return_index = True


#     # Hashing rows of states matrix:
#     hashed = torch.sum(hash_vec * states, dim=1) # Compute hashes.
#         # It is same as matrix product torch.matmul(hash_vec , states )
#         # but pay attention: such code work with GPU for integers
#         # While torch.matmul - does not work for GPU for integer data types,
#         # since old GPU hardware (before 2020: P100, T4) does not support integer matrix multiplication

#     # Sort
#     hashed_sorted, idx = torch.sort(hashed)
#     # Mask selects elements which are different from the consequite - that is unique elements (since vector is sorted on the previous step)
#     mask = torch.concat((torch.tensor([True], device = device), hashed_sorted[1:] - hashed_sorted[:-1] > 0))
#     return states[idx][mask]
def beam_search1(state_start, model, beam_width =  10_000, i_step_max = 400, verbose = 0, state_size=state_size):
    '''
    Finds a path from the state_start to the destination state.

    '''
    # Initialize array of states
    print(state_size)
    array_of_states = state_start.view(1, state_size  ).clone().to(dtype).to(device)
    state_destination = torch.arange( state_size, device=device, dtype = dtype)
    # i_step_max = 100
    for i_step in range(1,i_step_max+1):

        # Technical preparation: to apply permutations to array we need the trick:
        # naively it is: array[ :, IX_array], but actually we need  to write array[ range(N)[:, np.newaxis], IX_array  ]
        row_indices = np.arange( array_of_states.shape[0] )[:, np.newaxis]

        # Apply all moves to all current states, all new states saved in array_of_states_new
        array_of_states_new = torch.empty( (0,array_of_states.shape[1]) , device=device, dtype = dtype)
        for move in list_generators:
            array_of_states_tmp = array_of_states[row_indices,move]
            array_of_states_new = torch.concatenate([array_of_states_new, array_of_states_tmp],axis = 0)

        # Take only unique states
        # surprise: THAT IS CRITICAL for beam search performance !!!!
        # if that is not done - beam search  will not find the desired state - quite often
        # The reason - essentianlly beam can degrade, i.e. can be populated by copy of only one state
        # It is surprising that such degradation  happens quite often even for beam_width = 10_000 - but it is indeed so
        array_of_states_new = get_unique_states(array_of_states_new)

        # Check destination state found
        vec_tmp = torch.all(array_of_states_new == state_destination, axis =1) # Compare state_destination and each row array_of_states
        # print(torch.sum(array_of_states_new == state_destination, axis =1).max())
        flag_found_destination = torch.any(vec_tmp).item() # Check for coincidence
        if flag_found_destination:
            if verbose >= 1:
                print('Found destination state. ', 'i_step:', i_step, ' n_ways:', (vec_tmp).sum())
            break

        # ML-model inference - estimate distance of new states to the destination state
        t0 = time.time()
        if array_of_states_new.shape[0] > beam_width: # If we have not so many states - we take them all - no need for ML-model

            # ML-model inference - estimate distance of new states to the destination state
#             y_pred = model.predict(array_of_states_new.cpu().numpy() )
            y_pred = make_predictions_on_array(model, array_of_states_new.cpu().numpy() )
            # Take only "beam_width" of the best states (i.e. most nearest to destination according to the model estimate)
            idx = np.argsort(y_pred)[:beam_width]
            array_of_states = array_of_states_new[idx,:]

        else:
            array_of_states = array_of_states_new
        predict_time = time.time() - t0
        if verbose >= 10:
          if i_step % 100 == 0:
            print(torch.sum(array_of_states_new == state_destination, axis =1).max())
            print(i_step,'i_step', '%.3f'%predict_time, 'predict_time', array_of_states_new.shape, 'array_of_states_new.shape' )

    if verbose >= 1:
        print();
        print('Search finished.', 'beam_width:', beam_width)
        if flag_found_destination:
            print(i_step, ' steps to destination state. Path found.')
        else:
            print('Path not found.')

    return flag_found_destination, i_step

def run_beam_search1(beam_width=256*32*4,i_step_max=1000):
  state_size = len(list_generators[0])
  hash_vec = torch.randint(-(2**60), 2**60, (state_size,), dtype = torch.int64, device = device )
  print(hash_vec.shape)
  n_scrambles_starting_state = 5
  n_gens = len(list_generators)
  state_destination = torch.arange( state_size, device=device, dtype = dtype)
  # Scramble - generate state which will be start of beam search
  state_start = state_destination
  # for k in range(n_scrambles_starting_state):
  #     IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
  #     state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ]
  state_start = scramble_given_state_custom(list_generators, 1, state_start)
  print(state_start.shape)

  flag_found_destination, i_step = beam_search1(state_start, model, beam_width = beam_width,
                                                i_step_max = 10000,
                                              verbose = 100, state_size=state_size)


In [ ]:
# run_beam_search1()

In [ ]:
run_beam_search1(beam_width=131072)

torch.Size([16])
torch.Size([16])
16
tensor(11, device='cuda:0')
100 i_step 4.422 predict_time torch.Size([277215, 16]) array_of_states_new.shape
Found destination state.  i_step: 128  n_ways: tensor(1, device='cuda:0')

Search finished. beam_width: 131072
128  steps to destination state. Path found.


In [ ]:
run_beam_search1(beam_width=131072*4)

torch.Size([16])
torch.Size([16])
16
tensor(13, device='cuda:0')
100 i_step 17.391 predict_time torch.Size([1083078, 16]) array_of_states_new.shape
Found destination state.  i_step: 122  n_ways: tensor(1, device='cuda:0')

Search finished. beam_width: 524288
122  steps to destination state. Path found.


In [ ]:
sequence_length, beam_width, steps
10, 1000, 45
16, 32768, 132
16, 131072, 128
16, 524288, 122

40

In [ ]:
dimension = 40
dic={5: 20,
     10: 140,
     16: 180,
    20:	250,
21:	273,
22:	297,
23:	322,
24:	348,
25:	375,
26:	403,
27:	432,
28:	462,
29:	493,
30:	525,
31:	558,
32:	592,
35:	700,
37:	777,
40:	900,
45:	1125,
50:	1375,
60:	1950,
70:	2625,
80:	3400,
90:	4275,
100: 5250,
150:	11625,
200:	20500,
300:	45750}
n_random_walk_length = dic[dimension]
rate_of_diffusion_distance =10
n_random_walk_length = n_random_walk_length*rate_of_diffusion_distance
n_random_walks_to_generate_validation_fixed = 100
# n_random_walk_length = 40
n_random_walks_to_generate = 1000

def make_S_N(N):
    return [
        [1,0,]+[q+2 for q in range(N-2)]       ,
               [q+1 for q in range(N-1)] + [0,],
        [N-1,]+[q+0 for q in range(N-1)]       ,
    ]
list_generators = make_S_N(dimension)
X_test, y_test = random_walks(n_random_walk_length       = n_random_walk_length,
                                                     n_random_walks_to_generate = n_random_walks_to_generate)
X_test.shape

torch.Size([9000000, 40])

In [ ]:
class ResidualBlock1D(nn.Module):
    """
    A single residual block for 1D signals:
      - Conv1D -> BN -> ReLU -> Dropout -> Conv1D -> BN
      - skip connection
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, dropout=0.3):
        super().__init__()
        padding = kernel_size // 2  # same padding
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding)
        self.bn1   = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, padding=padding)
        self.bn2   = nn.BatchNorm1d(out_channels)

        # If in_channels != out_channels, we need a 1x1 conv to match dimensions for the skip
        self.shortcut = None
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = self.bn2(out)

        # Match channels if needed
        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        out = F.relu(out)
        return out

class ResNet1D(nn.Module):
    """
    A ResNet-style 1D CNN for sequences of variable length:
      1) Embedding (optional, if inputs are integer tokens)
      2) Initial Conv1D
      3) Multiple ResidualBlock1D layers
      4) Global AdaptiveAvgPool1d(1)
      5) Fully-connected output
    """
    def __init__(self,
                 vocab_size=100,
                 embed_dim=32,
                 hidden_dims=[64, 128, 128],
                 out_dim=1,
                 dropout=0.3):
        super().__init__()
        self.out_dim = out_dim

        # Embedding: for integer input sequences
        self.embedding = nn.Embedding(vocab_size, embed_dim)

        # Initial projection from embed_dim -> hidden_dims[0]
        self.input_conv = nn.Conv1d(embed_dim, hidden_dims[0], kernel_size=3, padding=1)
        self.bn_input   = nn.BatchNorm1d(hidden_dims[0])

        # Create a stack of residual blocks
        blocks = []
        for i in range(len(hidden_dims) - 1):
            in_ch  = hidden_dims[i]
            out_ch = hidden_dims[i+1]
            block  = ResidualBlock1D(in_ch, out_ch, kernel_size=3, dropout=dropout)
            blocks.append(block)
        self.res_blocks = nn.Sequential(*blocks)

        # Global pooling to get a single vector
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        # Final linear layer
        self.fc = nn.Linear(hidden_dims[-1], out_dim)

    def forward(self, x):
        """
        x: (batch_size, seq_len) with integer tokens in [0..vocab_size-1]
        """
        # 1) Embed => (batch, seq_len, embed_dim)
        x = self.embedding(x)
        # 2) Permute => (batch, embed_dim, seq_len)
        x = x.permute(0, 2, 1)

        # 3) Input conv -> BN -> ReLU
        x = self.input_conv(x)
        x = self.bn_input(x)
        x = F.relu(x)

        # 4) Pass through residual blocks
        x = self.res_blocks(x)  # shape still (batch, hidden_dims[-1], seq_len)

        # 5) Global pooling => (batch, hidden_dims[-1], 1)
        x = self.global_pool(x)
        x = x.squeeze(-1)       # => (batch, hidden_dims[-1])

        # 6) Final FC => (batch, out_dim)
        out = self.fc(x)
        if self.out_dim == 1:
            out = out.squeeze(-1)  # scalar for each batch element
        return out

In [ ]:

# model = ResNet1D(vocab_size=dimension, embed_dim=64, hidden_dims=[64, 128, 128, 256, 256]).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.8, verbose=True)
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
# print(f'Total trainable parameters: {pytorch_total_params}')

Total trainable parameters: 0.921409


In [ ]:

# model = ResNet1D(vocab_size=dimension, embed_dim=128, hidden_dims=[64, 128, 128, 256, 256, 512, 512]).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.8, verbose=True)
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
# print(f'Total trainable parameters: {pytorch_total_params}')

Total trainable parameters: 3.826753


In [ ]:
test_dataset = MyIntSequenceDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# !ls /content/drive/MyDrive/archive/

In [ ]:
best_test_rmse = '1028.5249'
model_path = f'/content/drive/MyDrive/archive/best_model_1028.52485385624.pth'
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list = train(100)

X.shape=torch.Size([9000000, 40])
Epoch [1/100], Loss: 53044.4336
Train - RMSE: 181.6841, R²: 0.9951
Test  - RMSE: 1088.3258, R²: 0.8245
Epoch [2/100], Loss: 42284.2578
Train - RMSE: 157.3925, R²: 0.9963
Test  - RMSE: 1069.0997, R²: 0.8307
Epoch [3/100], Loss: 39155.0234
Train - RMSE: 149.4759, R²: 0.9967
Test  - RMSE: 1056.1937, R²: 0.8347
Epoch [4/100], Loss: 34024.4414
Train - RMSE: 188.0088, R²: 0.9948
Test  - RMSE: 1048.3807, R²: 0.8372
Epoch [5/100], Loss: 26315.2012
Train - RMSE: 167.2942, R²: 0.9959
Test  - RMSE: 1054.4815, R²: 0.8353
Epoch [6/100], Loss: 72281.7500
Train - RMSE: 182.3396, R²: 0.9951
Test  - RMSE: 1046.6954, R²: 0.8377
Epoch [7/100], Loss: 65036.2891
Train - RMSE: 167.0597, R²: 0.9959
Test  - RMSE: 1036.3456, R²: 0.8409
Epoch [8/100], Loss: 34897.2969
Train - RMSE: 170.4840, R²: 0.9957
Test  - RMSE: 1049.6358, R²: 0.8368
Epoch [9/100], Loss: 32782.3594
Train - RMSE: 166.0198, R²: 0.9959
Test  - RMSE: 1040.1753, R²: 0.8397
Epoch [10/100], Loss: 32705.2930
Train 

In [ ]:
1

1

In [ ]:
# state_size = len(list_generators[0])
# hash_vec = torch.randint(-(2**60), 2**60, (state_size,), dtype = torch.int64, device = device )
# # hash_vec = torch.randint(0, 10**9, (state_size,), dtype = torch.int64, device = device )

# def get_unique_states(states: torch.Tensor) -> torch.Tensor:
#     '''
#     Return matrix with unique rows for input matrix "states"
#     I.e. duplicate rows are dropped.
#     For fast implementation: we use hashing via scalar/dot product.
#     Note: output order of rows is different from the original.
#     '''
#     # Note: that implementation is 30 times faster than torch.unique(states, dim = 0) - because we use hashes  (see K.Khoruzhii: https://t.me/sberlogasci/10989/15920)
#     # Note: torch.unique does not support returning of indices of unique element so we cannot use it
#     # That is in contrast to numpy.unique which supports - set: return_index = True


#     # Hashing rows of states matrix:
#     hashed = torch.sum(hash_vec * states, dim=1) # Compute hashes.
#         # It is same as matrix product torch.matmul(hash_vec , states )
#         # but pay attention: such code work with GPU for integers
#         # While torch.matmul - does not work for GPU for integer data types,
#         # since old GPU hardware (before 2020: P100, T4) does not support integer matrix multiplication

#     # Sort
#     hashed_sorted, idx = torch.sort(hashed)
#     # Mask selects elements which are different from the consequite - that is unique elements (since vector is sorted on the previous step)
#     mask = torch.concat((torch.tensor([True], device = device), hashed_sorted[1:] - hashed_sorted[:-1] > 0))
#     return states[idx][mask]
# def beam_search1(state_start, model, beam_width =  10_000, i_step_max = 400, verbose = 0, state_size=state_size):
#     '''
#     Finds a path from the state_start to the destination state.

#     '''
#     # Initialize array of states
#     print(state_size)
#     array_of_states = state_start.view(1, state_size  ).clone().to(dtype).to(device)
#     state_destination = torch.arange( state_size, device=device, dtype = dtype)
#     # i_step_max = 100
#     for i_step in range(1,i_step_max+1):

#         # Technical preparation: to apply permutations to array we need the trick:
#         # naively it is: array[ :, IX_array], but actually we need  to write array[ range(N)[:, np.newaxis], IX_array  ]
#         row_indices = np.arange( array_of_states.shape[0] )[:, np.newaxis]

#         # Apply all moves to all current states, all new states saved in array_of_states_new
#         array_of_states_new = torch.empty( (0,array_of_states.shape[1]) , device=device, dtype = dtype)
#         for move in list_generators:
#             array_of_states_tmp = array_of_states[row_indices,move]
#             array_of_states_new = torch.concatenate([array_of_states_new, array_of_states_tmp],axis = 0)

#         # Take only unique states
#         # surprise: THAT IS CRITICAL for beam search performance !!!!
#         # if that is not done - beam search  will not find the desired state - quite often
#         # The reason - essentianlly beam can degrade, i.e. can be populated by copy of only one state
#         # It is surprising that such degradation  happens quite often even for beam_width = 10_000 - but it is indeed so
#         array_of_states_new = get_unique_states(array_of_states_new)

#         # Check destination state found
#         vec_tmp = torch.all(array_of_states_new == state_destination, axis =1) # Compare state_destination and each row array_of_states
#         # print(torch.sum(array_of_states_new == state_destination, axis =1).max())
#         flag_found_destination = torch.any(vec_tmp).item() # Check for coincidence
#         if flag_found_destination:
#             if verbose >= 1:
#                 print('Found destination state. ', 'i_step:', i_step, ' n_ways:', (vec_tmp).sum())
#             break

#         # ML-model inference - estimate distance of new states to the destination state
#         t0 = time.time()
#         if array_of_states_new.shape[0] > beam_width: # If we have not so many states - we take them all - no need for ML-model

#             # ML-model inference - estimate distance of new states to the destination state
# #             y_pred = model.predict(array_of_states_new.cpu().numpy() )
#             y_pred = make_predictions_on_array(model, array_of_states_new.cpu().numpy() )
#             # Take only "beam_width" of the best states (i.e. most nearest to destination according to the model estimate)
#             idx = np.argsort(y_pred)[:beam_width]
#             array_of_states = array_of_states_new[idx,:]

#         else:
#             array_of_states = array_of_states_new
#         predict_time = time.time() - t0
#         if verbose >= 10:
#           if i_step % 100 == 0:
#             print(torch.sum(array_of_states_new == state_destination, axis =1).max())
#             print(i_step,'i_step', '%.3f'%predict_time, 'predict_time', array_of_states_new.shape, 'array_of_states_new.shape' )

#     if verbose >= 1:
#         print();
#         print('Search finished.', 'beam_width:', beam_width)
#         if flag_found_destination:
#             print(i_step, ' steps to destination state. Path found.')
#         else:
#             print('Path not found.')

#     return flag_found_destination, i_step

# def run_beam_search1(beam_width=256*32*4):
#     # %%time
#   state_size = len(list_generators[0])
#   hash_vec = torch.randint(-(2**60), 2**60, (state_size,), dtype = torch.int64, device = device )
#   print(hash_vec.shape)
#   n_scrambles_starting_state = 5
#   n_gens = len(list_generators)
#   state_destination = torch.arange( state_size, device=device, dtype = dtype)
#   # Scramble - generate state which will be start of beam search
#   state_start = state_destination
#   # for k in range(n_scrambles_starting_state):
#   #     IX_move = np.random.randint(0, n_gens, dtype = int) # random moves indixes
#   #     state_start = state_start[ list_generators[IX_move]] # all_moves[IX_moves,:] ]
#   state_start = scramble_given_state_custom(list_generators, 1, state_start)
#   print(state_start.shape)

#   flag_found_destination, i_step = beam_search1(state_start, model, beam_width = beam_width,
#                                                 i_step_max = 1000,
#                                               verbose = 100, state_size=state_size)


In [ ]:
run_beam_search1(beam_width=10)

torch.Size([20])
torch.Size([20])
20
tensor(2, device='cuda:0')
100 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
200 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
300 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
400 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
500 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
600 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
700 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
800 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')
900 i_step 0.003 predict_time torch.Size([24, 20]) array_of_states_new.shape
tensor(2, device='cuda:0')


In [ ]:
run_beam_search1(beam_width=131072)

torch.Size([20])
torch.Size([20])
20


In [ ]:
def make_predictions_on_array(model, array):
    model.eval()
    all_preds = []
    dataset = MyIntSequenceDatasetSmall(array)
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    with torch.no_grad():
        for batch_X in loader:
            batch_X = batch_X.to(device)
            outputs = model(batch_X)
            all_preds.extend(outputs.cpu().numpy())

    all_preds = np.array(all_preds)

    return all_preds

In [ ]:
from collections import deque

def beam_search_sorting(permutation, beam_width=5):
    n = len(permutation)
    target = list(range(0, n))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def average_neighbor_difference(perm):
        total_diff = 0

        # Считаем разницы между соседними элементами
        for i in range(n-1):
            total_diff += abs(perm[i] - perm[i+1])

        # Добавляем разницу между последним и первым элементом
        total_diff += abs(perm[-1] - perm[0])

        return total_diff / n

    def count_monotonic_sections(perm):
        # Создаем расширенную перестановку для учета цикличности
        extended_perm = perm + perm[0:1]

        sections = 0
        increasing = None

        for i in range(n):
            if increasing is None:  # Первое сравнение
                increasing = extended_perm[i] < extended_perm[i + 1]
            elif increasing and extended_perm[i] > extended_perm[i + 1]:  # Смена возрастания на убывание
                sections += 1
                increasing = False
            elif not increasing and extended_perm[i] < extended_perm[i + 1]:  # Смена убывания на возрастание
                sections += 1
                increasing = True

        return sections / n

    def apply_actions(state):
        # Generate new states based on the allowed actions
        states = []
        # 1. Cyclic shift right
        new_state = state[-1:] + state[:-1]
        states.append((new_state, "R"))

        # 2. Cyclic shift left
        new_state = state[1:] + state[:1]
        states.append((new_state, "L"))

        # 3. Swap first two elements
        if len(state) > 1:
            new_state = state[:]
            new_state[0], new_state[1] = new_state[1], new_state[0]
            states.append((new_state, "X"))

        return states

    def apply_action(state, action):
        # 1. Cyclic shift right
        if action == 'R':
            new_state = state[-1:] + state[:-1]
            return new_state
        elif action == 'L':
            # 2. Cyclic shift left
            new_state = state[1:] + state[:1]
            return new_state

        new_state = state[:]
        new_state[0], new_state[1] = new_state[1], new_state[0]
        return new_state


    # Priority queue for the beam search; stores tuples of (cumulative cost, path, current state)
    queue = deque([(0, [], permutation)])
    seen = set()

    max_actions = 0
    q_values = []

    while queue:
        # Limit the size of the queue as per beam width
        queue = deque(sorted(list(queue), key=lambda x: x[0])[:beam_width])
        next_queue = deque()

        for cost, path, current in queue:
            if len(path) > max_actions:
                max_actions = len(path)
                if max_actions % 100 == 0:
                    print('new max len', len(path), len(queue), len(seen), np.min(q_values))
                q_values = []
            if len(path) > 40000:
                #print('long path', queue)
                return None
            if current == target:
                return path  # Return the path to sorted order


            for action in ['L','R','X']:
                if action == 'L' and len(path) > 0 and path[-1] == 'R':
                    continue
                if action == 'R' and len(path) > 0 and path[-1] == 'L':
                    continue
                if action == 'X' and current[0] < current[1]:
                    continue

                next_state = apply_action(current, action)

                if tuple(next_state) not in seen:
                    seen.add(tuple(next_state))

                    with torch.no_grad():
                        t_state = torch.FloatTensor(next_state).unsqueeze(0).to(device)
                        q_value = make_predictions_on_array(model, t_state.cpu().numpy() )
                        # q_value = model(t_state).squeeze().item()
                        q_values.append(q_value)

                    total_cost = cost + 1 + q_value
                    next_queue.append((total_cost, path + [action], next_state))

        queue = next_queue

    return None  # Return None if no solution is found

# Пример использования
p = np.arange(dimension)
n = dimension
p[0], p[1] = p[1], p[0]
i = 2
while i < n-i+1:
    p[i], p[n-i+1] = p[n-i+1], p[i]
    i += 1
permutation = p.tolist()

# k = 10  # Ширина "луча"
# model.eval();
# path = beam_search_sorting(permutation, beam_width=k)
# if path:
#   print("Кратчайший путь:", len(path))
#   print(path)
# else:
#   print('No path found')


In [ ]:
# k = 100
# path = beam_search_sorting(permutation, beam_width=k)
# if path:
#   print("Кратчайший путь:", len(path))
#   print(path)
# else:
#   print('No path found')

In [ ]:
k = 1000
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 1000 160346 3098.1602
new max len 200 1000 323851 3279.9788
new max len 300 1000 482047 3461.4033
new max len 400 1000 630470 3728.8447
new max len 500 1000 769213 3841.1826
new max len 600 1000 910599 4147.19
new max len 700 1000 1046679 3923.6255
new max len 800 1000 1185759 4047.1436
new max len 900 1000 1336775 4105.2744
new max len 1000 1000 1481288 3311.1409
new max len 1100 1000 1632446 3189.753
new max len 1200 1000 1776717 1575.9518
new max len 1300 1000 1908387 1542.0941
new max len 1400 1000 2035682 1760.7522
new max len 1500 1000 2161028 1240.5491


KeyboardInterrupt: 

In [ ]:
dimension

20

In [ ]:
#20 - fastest 246.


In [ ]:
1

1

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GatedResidualBlock1D(nn.Module):
    """
    A 1D TCN block with gated activation:
      - Conv1D (dilation) -> BN -> [sigmoid gate * tanh candidate] -> Dropout -> Conv1D ...
      - Residual skip
    """
    def __init__(self, in_channels, out_channels, kernel_size=3, dilation=1, dropout=0.3):
        super().__init__()
        self.out_channels = out_channels
        pad = (kernel_size - 1) * dilation // 2

        # First conv for the gated activation
        self.conv_filter = nn.Conv1d(in_channels, out_channels, kernel_size,
                                     padding=pad, dilation=dilation)
        self.conv_gate   = nn.Conv1d(in_channels, out_channels, kernel_size,
                                     padding=pad, dilation=dilation)
        self.bn_filter = nn.BatchNorm1d(out_channels)
        self.bn_gate   = nn.BatchNorm1d(out_channels)
        self.dropout = nn.Dropout(dropout)

        # Second conv
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=pad, dilation=dilation)
        self.bn2   = nn.BatchNorm1d(out_channels)

        self.shortcut = None
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, 1)

    def forward(self, x):
        residual = x

        # Gated activation
        filter_out = self.conv_filter(x)
        filter_out = self.bn_filter(filter_out)
        gate_out   = self.conv_gate(x)
        gate_out   = self.bn_gate(gate_out)

        filter_out = torch.tanh(filter_out)
        gate_out   = torch.sigmoid(gate_out)
        out = filter_out * gate_out  # gating

        out = self.dropout(out)

        # Second conv
        out = self.conv2(out)
        out = self.bn2(out)

        # Residual
        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        out = F.relu(out)
        return out


class GatedTCN1D(nn.Module):
    """
    An improved TCN for 1D sequences using gated residual blocks.
      1) Embedding (if needed)
      2) Stacked GatedResidualBlock1D with dilations 1, 2, 4, ...
      3) Global pool
      4) FC for output
    """
    def __init__(self,
                 vocab_size=50,
                 embed_dim=32,
                 hidden_dim=128,
                 num_levels=4,
                 kernel_size=3,
                 dropout=0.3,
                 out_dim=1):
        super().__init__()
        self.out_dim = out_dim

        # Embedding for integer tokens
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.proj = nn.Conv1d(embed_dim, hidden_dim, kernel_size=1)

        # Build stacked blocks with growing dilation
        self.blocks = nn.ModuleList()
        dilation = 1
        for _ in range(num_levels):
            block = GatedResidualBlock1D(
                in_channels=hidden_dim,
                out_channels=hidden_dim,
                kernel_size=kernel_size,
                dilation=dilation,
                dropout=dropout
            )
            self.blocks.append(block)
            dilation *= 2

        # Global pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        # Final linear
        self.fc = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        """
        x: (batch_size, seq_len) integer tokens
        """
        # 1) Embedding -> (batch, seq_len, embed_dim)
        x = self.embedding(x)
        # 2) => (batch, embed_dim, seq_len)
        x = x.permute(0, 2, 1)

        # 3) Project to hidden_dim
        x = self.proj(x)

        # 4) Pass through each gated TCN block
        for block in self.blocks:
            x = block(x)

        # 5) Global pool => (batch, hidden_dim, 1)
        x = self.global_pool(x)
        x = x.squeeze(-1)  # => (batch, hidden_dim)

        # 6) Output
        out = self.fc(x)
        if self.out_dim == 1:
            out = out.squeeze(-1)
        return out

# # ----------------------------------
# # EXAMPLE USAGE
# # ----------------------------------
# if __name__ == "__main__":
#     batch_size = 4
#     seq_len = 16
#     X = torch.randint(0, 50, (batch_size, seq_len))
#     y = torch.randn(batch_size)

#     model = GatedTCN1D(vocab_size=50, embed_dim=32, hidden_dim=128,
#                        num_levels=4, kernel_size=3, dropout=0.3, out_dim=1)
#     print(model)
#     pred = model(X)
#     print("Output shape:", pred.shape)

#     criterion = nn.MSELoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#     loss = criterion(pred, y)
#     loss.backward()
#     optimizer.step()
#     print("Loss:", loss.item())

NameError: name 'X_train' is not defined

In [ ]:

# model = GatedTCN1D(vocab_size=40, embed_dim=32, hidden_dim=128,
#                        num_levels=4, kernel_size=3, dropout=0.3, out_dim=1).to(device)
# criterion = nn.MSELoss()
# optimizer = optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.8, verbose=True)
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
# print(f'Total trainable parameters: {pytorch_total_params}')
# model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list = train(100)

Total trainable parameters: 0.600065
X.shape=torch.Size([9000000, 40])
Epoch [1/100], Loss: 154755.9375
Train - RMSE: 292.4204, R²: 0.9873
Test  - RMSE: 1084.7953, R²: 0.8257
Epoch [2/100], Loss: 97282.0547
Train - RMSE: 286.6115, R²: 0.9878
Test  - RMSE: 1082.7522, R²: 0.8263
Epoch [3/100], Loss: 76442.2031
Train - RMSE: 372.5493, R²: 0.9794
Test  - RMSE: 1116.3921, R²: 0.8154
Epoch [4/100], Loss: 58801.4375
Train - RMSE: 351.3758, R²: 0.9817
Test  - RMSE: 1116.1797, R²: 0.8154
Epoch [5/100], Loss: 82697.8203
Train - RMSE: 402.1689, R²: 0.9760
Test  - RMSE: 1134.9644, R²: 0.8092
Epoch [6/100], Loss: 72408.7031
Train - RMSE: 420.1852, R²: 0.9738
Test  - RMSE: 1130.4998, R²: 0.8107
Epoch [7/100], Loss: 95541.1250
Train - RMSE: 390.9566, R²: 0.9774
Test  - RMSE: 1102.8646, R²: 0.8198
Early stopping triggered
Finished Training
Best metrics: min(test_rmse_list)=1082.7522 max(test_r2_list)=0.8263


In [ ]:
k = 10  # Ширина "луча"
model.eval();
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 10 1723 7660.8794
new max len 200 10 3453 7051.2144
new max len 300 10 5017 7110.5254
new max len 400 10 6606 6834.5244
new max len 500 10 8174 5918.989
new max len 600 10 9700 6091.792
new max len 700 10 11196 5626.2476
new max len 800 10 12633 5451.3745
new max len 900 10 14050 4583.1235
new max len 1000 10 15480 3416.4167
new max len 1100 10 16817 2520.675
new max len 1200 10 18127 2888.3318
new max len 1300 10 19463 1795.1128
new max len 1400 10 20759 335.62872
new max len 1500 10 22077 137.15617
new max len 1600 10 23292 53.02608
new max len 1700 10 24520 189.29395
new max len 1800 10 25696 251.09688
new max len 1900 10 26947 107.45617
new max len 2000 10 28205 339.42084
new max len 2100 10 29426 827.9243
new max len 2200 10 30638 1172.1304
new max len 2300 10 31903 1555.2037
new max len 2400 10 33232 1858.5203
new max len 2500 10 34590 2150.0344
new max len 2600 10 35911 2393.0952
new max len 2700 10 37289 1510.8402
new max len 2800 10 38709 1041.2988
new max len 

In [ ]:
k = 100  # Ширина "луча"
model.eval();
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 100 16628 6860.6694
new max len 200 100 32786 6867.855
new max len 300 100 47828 6592.356
new max len 400 100 61995 6379.0015
new max len 500 100 76290 5533.2944
new max len 600 100 91544 5528.895
new max len 700 100 107269 4944.6616
new max len 800 100 122454 3759.0889


KeyboardInterrupt: 

In [ ]:
# 32496 -> 21322

In [ ]:

model = GatedTCN1D(vocab_size=dimension, embed_dim=64, hidden_dim=256,
                       num_levels=6, kernel_size=3, dropout=0.3, out_dim=1).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.8, verbose=True)
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000
print(f'Total trainable parameters: {pytorch_total_params}')
model, train_rmse_list, train_r2_list, test_rmse_list, test_r2_list, loss_list = train(100)

Total trainable parameters: 3.572225
X.shape=torch.Size([9000000, 40])
Epoch [1/100], Loss: 236392.3750
Train - RMSE: 512.7828, R²: 0.9610
Test  - RMSE: 1439.8027, R²: 0.6929
Epoch [2/100], Loss: 110769.2344
Train - RMSE: 306.8331, R²: 0.9861
Test  - RMSE: 1154.5050, R²: 0.8025
Epoch [3/100], Loss: 133935.4062
Train - RMSE: 273.0778, R²: 0.9890
Test  - RMSE: 1160.4948, R²: 0.8005
Epoch [4/100], Loss: 245607.5625
Train - RMSE: 125.7377, R²: 0.9977
Test  - RMSE: 1052.6852, R²: 0.8358
Epoch [5/100], Loss: 86071.2031
Train - RMSE: 140.8219, R²: 0.9971
Test  - RMSE: 1066.4022, R²: 0.8315
Epoch [6/100], Loss: 38557.3828
Train - RMSE: 166.7094, R²: 0.9959
Test  - RMSE: 1071.3007, R²: 0.8300
Epoch [7/100], Loss: 44742.1445
Train - RMSE: 161.2146, R²: 0.9961
Test  - RMSE: 1060.5569, R²: 0.8334
Epoch [8/100], Loss: 66082.3906
Train - RMSE: 184.9591, R²: 0.9949
Test  - RMSE: 1069.3910, R²: 0.8306
Epoch [9/100], Loss: 56821.6172
Train - RMSE: 212.3311, R²: 0.9933
Test  - RMSE: 1087.2189, R²: 0.824

In [ ]:
k = 10  # Ширина "луча"
model.eval();
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 10 1799 8215.915
new max len 200 10 3516 7810.1367
new max len 300 10 5133 7484.3584
new max len 400 10 6692 7276.143
new max len 500 10 8110 6937.1484
new max len 600 10 9622 6068.4307
new max len 700 10 11230 5391.6074
new max len 800 10 12763 4407.8887
new max len 900 10 14306 4307.674
new max len 1000 10 15767 3544.4795
new max len 1100 10 17272 1985.5748
new max len 1200 10 18718 953.8212
new max len 1300 10 20084 396.62567
new max len 1400 10 21427 89.65303
new max len 1500 10 22667 61.10315
new max len 1600 10 23916 164.61758
new max len 1700 10 25163 316.60645
new max len 1800 10 26446 168.43951
new max len 1900 10 27634 115.68915
new max len 2000 10 28864 203.70316
new max len 2100 10 30082 86.825325
new max len 2200 10 31283 150.45686
new max len 2300 10 32510 181.21222
new max len 2400 10 33760 242.73027
new max len 2500 10 34997 156.04831
new max len 2600 10 36192 323.12457
new max len 2700 10 37408 393.46118
new max len 2800 10 38669 289.92236
new max len 2

In [ ]:
k = 20  # Ширина "луча"
model.eval();
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 20 3486 8082.0635
new max len 200 20 6669 7899.4937
new max len 300 20 9919 7682.0117
new max len 400 20 13022 7327.7134
new max len 500 20 15966 6892.7847
new max len 600 20 19114 6124.3315
new max len 700 20 22362 5609.128
new max len 800 20 25452 5272.619
new max len 900 20 28475 3171.8206
new max len 1000 20 31501 1794.8247
new max len 1100 20 34493 816.8391
new max len 1200 20 37464 132.63293
new max len 1300 20 39949 52.83428
new max len 1400 20 42277 109.14501
new max len 1500 20 44715 55.21481
new max len 1600 20 47152 176.7174
new max len 1700 20 49622 256.03354
new max len 1800 20 52088 432.80756
new max len 1900 20 54549 517.4871
new max len 2000 20 57094 360.45673
new max len 2100 20 59566 196.39426
new max len 2200 20 62025 153.62598
new max len 2300 20 64445 111.65724
new max len 2400 20 66954 198.15662
new max len 2500 20 69422 162.65097
new max len 2600 20 72070 316.70905
new max len 2700 20 74721 179.25566
new max len 2800 20 77383 103.07468
new max len

In [ ]:
k = 100  # Ширина "луча"
model.eval();
path = beam_search_sorting(permutation, beam_width=k)
if path:
  print("Кратчайший путь:", len(path))
  print(path)
else:
  print('No path found')

new max len 100 100 16718 7881.874
new max len 200 100 32815 7703.749
new max len 300 100 48422 7548.4375
new max len 400 100 63727 7138.293
new max len 500 100 78555 6845.26
new max len 600 100 93463 6316.063
new max len 700 100 107844 4816.045
new max len 800 100 122221 3158.1777
new max len 900 100 137416 1685.951
new max len 1000 100 151420 545.7935
new max len 1100 100 165271 63.90102
Кратчайший путь: 1136
['R', 'X', 'R', 'R', 'X', 'L', 'X', 'L', 'L', 'X', 'L', 'L', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'X', 'L', 'X', 'R', 'X', 'L', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'X', 'L', 'L', 'X', 'L', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'X', 'L', 'X', 'L', 'X', 'L', 'L', 'X', 'L', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'L', 'L', 'L', 'L', 'X', 'R', 'X', 'L', 'X', 'L', 'X', 'L', 'X', 'L', 'L', 'X', 'L', 'X', 'R', 'X', 'L', 'L', 'L', 'L', 'L', 'L', 'X', 'L', 'L', 'L', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R', 'X', 'R',

In [ ]:
X_test.shape

torch.Size([9000000, 40])

In [ ]:
run_beam_search1(beam_width=131072, i_step_max=15000)

torch.Size([40])
torch.Size([40])
40
tensor(4, device='cuda:0')
100 i_step 15.515 predict_time torch.Size([274619, 40]) array_of_states_new.shape
tensor(7, device='cuda:0')
200 i_step 15.015 predict_time torch.Size([264724, 40]) array_of_states_new.shape
tensor(5, device='cuda:0')
300 i_step 14.703 predict_time torch.Size([260730, 40]) array_of_states_new.shape
tensor(5, device='cuda:0')
400 i_step 15.165 predict_time torch.Size([268095, 40]) array_of_states_new.shape
tensor(6, device='cuda:0')
500 i_step 15.556 predict_time torch.Size([272916, 40]) array_of_states_new.shape
tensor(6, device='cuda:0')
600 i_step 15.629 predict_time torch.Size([276511, 40]) array_of_states_new.shape


In [ ]:
1

1